In [1]:
from neurules import NeuralRuleList
from configs import *
from utils import get_data_limits, get_dataset, binarize_categorical, Temperature_Scheduler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from training import train_model
import torch

# load example dataset: credit_g
dataset, _ = get_dataset("credit_g")
X = dataset["data"]
X, dataset["feature_names"] = binarize_categorical(X, dataset["feature_names"])
Y = dataset["target"]
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)

# initialize NeuRules thresholds with data limits
init_thresholds = get_data_limits(X_train)

model_config = Rule_Config(X_train.shape[1],2,10)
model = NeuralRuleList(model_config, init_thresholds)

training_setup = Train_Config()
predicate_schedule = Temperature_Scheduler(training_setup.n_epochs//2,model_config.schedule_predicate_temperature)
selector_schedule = Temperature_Scheduler(training_setup.n_epochs//2,model_config.schedule_selector_temperature)
temp_schedule = [predicate_schedule, selector_schedule]
model = train_model(model,X_train,Y_train,torch.nn.CrossEntropyLoss(),training_setup)

# evaluate on test set
model.eval()
with torch.no_grad():
    Y_logits = model(X_test)
    Y_pred = Y_logits.argmax(dim=1)
    accuracy = (Y_pred == Y_test).float().mean().item()
    print(f"Test Accuracy: {accuracy*100:.2f}%")
    
    # print learned rule list
    limits = get_data_limits(X_train)
    rule_list = model.print_rule_list(limits, scaler_x=scaler_X, feature_names=dataset["feature_names"])
    print("Learned Rule List:")
    print(rule_list)

fetching credit_g from imodels


100%|██████████| 2000/2000 [00:33<00:00, 58.94it/s]

Test Accuracy: 68.50%
Learned Rule List:
Rule 1 with 6.52 for class 0 with likelihood 50.29%: 0.54 < checking_status_0.0 ∧ checking_status_3.0 < 0.79
Rule 8 with 6.15 for class 1 with likelihood 54.51%: 12.97 < duration ∧ credit_amount < 15188.02 ∧ age < 44.76 ∧ 0.56 < other_payment_plans_0.0 ∧ other_payment_plans_2.0 < 0.69
Rule 7 with 5.58 for class 1 with likelihood 85.85%: 30.80 < age ∧ credit_history_0.0 < 0.59 ∧ credit_history_1.0 < 0.59
Rule 4 with 5.52 for class 0 with likelihood 57.72%: 0.41 < credit_history_0.0
Rule 9 with 4.69 for class 1 with likelihood 54.57%: 0.55 < checking_status_0.0
Rule 5 with 3.12 for class 1 with likelihood 78.59%: 24.95 < age ∧ credit_history_0.0 < 0.66 ∧ credit_history_1.0 < 0.66
Rule 6 with 2.03 for class 1 with likelihood 66.00%: checking_status_0.0 < 0.55 ∧ 0.17 < credit_history_1.0
Rule 3 with 2.02 for class 1 with likelihood 70.66%: 0.05 < credit_history_0.0 < 0.74 ∧ 0.00 < credit_history_1.0 < 0.76
Rule 0 with 1.44 for class 1 with likelihoo